### Weighted boxes fusion

[github link](https://github.com/ZFTurbo/Weighted-Boxes-Fusion)

* NMS : Non-maximum Supression
* Soft-NMS
* NMW : Non-maximum weighted
* WBF : Weighted boxes fusion - new method which gives better results comparing to others

In [161]:
import ensemble_boxes
from ensemble_boxes import *
from tqdm import tqdm
import pandas as pd
import numpy as np
dir(ensemble_boxes)

['__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'ensemble_boxes_nms',
 'ensemble_boxes_nmw',
 'ensemble_boxes_wbf',
 'ensemble_boxes_wbf_3d',
 'nms',
 'nms_method',
 'non_maximum_weighted',
 'soft_nms',
 'weighted_boxes_fusion',
 'weighted_boxes_fusion_3d']

```py
boxes, scores, labels = weighted_boxes_fusion(
                                        boxes_list,
                                        scores_list,
                                        labels_list,
                                        weights=None,
                                        iou_thr=0.55,
                                        skip_box_thr=0.0,
                                        conf_type='avg',
                                        allows_overflow=False,
)

```

In [7]:
def weighted_boxes_fusion_code(boxes_list, 
                               scores_list, 
                               labels_list, 
                               weights=None, 
                               iou_thr=0.55, 
                               skip_box_thr=0.0, 
                               conf_type='avg', 
                               allows_overflow=False):
    '''
    :param boxes_list: list of boxes predictions from each model, each box is 4 numbers.
    It has 3 dimensions (models_number, model_preds, 4)
    Order of boxes: x1, y1, x2, y2. We expect float normalized coordinates [0; 1]
    :param scores_list: list of scores for each model
    :param labels_list: list of labels for each model
    :param weights: list of weights for each model. Default: None, which means weight == 1 for each model
    :param iou_thr: IoU value for boxes to be a match
    :param skip_box_thr: exclude boxes with score lower than this variable
    :param conf_type: how to calculate confidence in weighted boxes. 
        'avg': average value, 
        'max': maximum value, 
        'box_and_model_avg': box and model wise hybrid weighted average, 
        'absent_model_aware_avg': weighted average that takes into account the absent model.
    :param allows_overflow: false if we want confidence score not exceed 1.0

    :return: boxes: boxes coordinates (Order of boxes: x1, y1, x2, y2).
    :return: scores: confidence scores
    :return: labels: boxes labels
    '''

    if weights is None:
        weights = np.ones(len(boxes_list))
    if len(weights) != len(boxes_list):
        print('Warning: incorrect number of weights {}. Must be: {}. Set weights equal to 1.'.format(len(weights), len(boxes_list)))
        weights = np.ones(len(boxes_list))
    weights = np.array(weights)

    if conf_type not in ['avg', 'max', 'box_and_model_avg', 'absent_model_aware_avg']:
        print('Unknown conf_type: {}. Must be "avg", "max" or "box_and_model_avg", or "absent_model_aware_avg"'.format(conf_type))
        exit()
    
    # boxes(x,y,w,h) 들 중에, 이상한 값이 없는지 체크
    filtered_boxes = prefilter_boxes(boxes_list, scores_list, labels_list, weights, skip_box_thr)
    if len(filtered_boxes) == 0:
        return np.zeros((0, 4)), np.zeros((0,)), np.zeros((0,))

    overall_boxes = []
    for label in filtered_boxes:
        boxes = filtered_boxes[label]
        new_boxes = []
        weighted_boxes = []
        # Clusterize boxes
        for j in range(0, len(boxes)):
            index, best_iou = find_matching_box(weighted_boxes, boxes[j], iou_thr)
            if index != -1:
                new_boxes[index].append(boxes[j])
                weighted_boxes[index] = get_weighted_box(new_boxes[index], conf_type)
            else:
                new_boxes.append([boxes[j].copy()])
                weighted_boxes.append(boxes[j].copy())
        # Rescale confidence based on number of models and boxes
        for i in range(len(new_boxes)):
            clustered_boxes = np.array(new_boxes[i])
            if conf_type == 'box_and_model_avg':
                # weighted average for boxes
                weighted_boxes[i][1] = weighted_boxes[i][1] * len(clustered_boxes) / weighted_boxes[i][2]
                # identify unique model index by model index column
                _, idx = np.unique(clustered_boxes[:, 3], return_index=True)
                # rescale by unique model weights
                weighted_boxes[i][1] = weighted_boxes[i][1] *  clustered_boxes[idx, 2].sum() / weights.sum()

            elif conf_type == 'absent_model_aware_avg':
                # get unique model index in the cluster
                models = np.unique(clustered_boxes[:, 3]).astype(int)
                # create a mask to get unused model weights
                mask = np.ones(len(weights), dtype=bool)
                mask[models] = False
                # absent model aware weighted average
                weighted_boxes[i][1] = weighted_boxes[i][1] * len(clustered_boxes) / (weighted_boxes[i][2] + weights[mask].sum())
            elif not allows_overflow:
                weighted_boxes[i][1] = weighted_boxes[i][1] * min(weights.sum(), len(clustered_boxes)) / weights.sum()
            else:
                weighted_boxes[i][1] = weighted_boxes[i][1] * len(clustered_boxes) / weights.sum()
        overall_boxes.append(np.array(weighted_boxes))
    overall_boxes = np.concatenate(overall_boxes, axis=0)
    overall_boxes = overall_boxes[overall_boxes[:, 1].argsort()[::-1]]
    boxes = overall_boxes[:, 4:]
    scores = overall_boxes[:, 1]
    labels = overall_boxes[:, 0]
    return boxes, scores, labels

<function ensemble_boxes.ensemble_boxes_wbf.weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=None, iou_thr=0.55, skip_box_thr=0.0, conf_type='avg', allows_overflow=False)>

In [10]:
# example with siim sample prediction
kfold_ids = [['51759b5579bc_image', '65761e66de9f_image'],
             ['51759b5579bc_image', '65761e66de9f_image'],
             ['51759b5579bc_image', '65761e66de9f_image'],
             ['51759b5579bc_image', '65761e66de9f_image'],
             ['51759b5579bc_image', '65761e66de9f_image']]

kfold_predictions = [
 ['0 0.001079560025 1625 1891 2427 2416 0 0.001121519948 1937 1299 2394 1758 0 0.001185419969 384 501 1097 1655 0 0.001243590028 423 816 757 1214 0 0.001252169954 289 809 662 1365 0 0.001281740027 579 1909 1169 2416 0 0.001282689977 334 1317 774 1903 0 0.001399039989 2137 1389 2527 1843 0 0.001568789943 1915 1927 2705 2398 0 0.001731869997 2154 1305 2655 2090 0 0.001977920067 540 1631 1102 2344 0 0.002027509967 1648 1855 2739 2604 0 0.002063750057 106 1607 629 2453 0 0.002161029959 328 2380 1386 3093 0 0.002309799893 334 665 891 1516 0 0.002346039983 406 1504 640 1716 0 0.003021240002 1653 1250 2327 2030 0 0.003068919992 379 1389 946 2235 0 0.003293989925 262 761 718 1849 0 0.004394529853 2082 1172 2572 1879 0 0.004405979998 1870 731 2427 1432 0 0.004520419985 1648 683 2433 1957 0 0.004955289885 223 1861 1280 2984 0 0.008049010299 1753 1565 2627 2422 0 0.009658809751 217 1009 774 2332 0 0.010353099555 289 731 980 2205 0 0.01126859989 195 1909 1124 2580 0 0.017990099266 2132 870 2405 1148 0 0.024032600224 1959 803 2483 1836 0 0.055938698351 223 1595 991 2441 0 0.071472197771 1887 1293 2549 2006',
  '0 0.001008030027 1854 218 2180 496 0 0.001054759952 592 765 929 1433 0 0.001077649998 1935 1001 2566 1656 0 0.001135829953 1930 637 2120 837 0 0.001306529972 837 1028 1288 1402 0 0.001331330044 783 519 1218 1192 0 0.001547810039 1859 346 2234 1069 0 0.001607890008 1707 746 2201 1465 0 0.002017969964 984 965 1353 1452 0 0.002161029959 1968 1106 2413 1474 0 0.002372740069 1832 246 2174 719 0 0.0026378599 2087 810 2375 1270 0 0.002689359942 1707 296 2212 1329 0 0.003168110037 1033 264 1402 1038 0 0.00460051978 902 724 1370 1393 0 0.005111690145 891 150 1424 1238 0 0.005161290057 761 978 1196 1324 0 0.005950930063 625 988 1283 1552 0 0.009033200331 674 637 1044 1356 0 0.009437560104 2104 906 2435 1488 0 0.010429400019 652 951 1049 1424 0 0.010986300185 2044 564 2430 1402 0 0.013572700322 1783 1033 2364 1570 0 0.030456500128 734 268 1277 1374 0 0.038452100009 1859 282 2381 1402 0 0.163330003619 707 810 1245 1397 0 0.194580003619 1892 801 2403 1470'],
 ['0 0.001277919975 362 1933 1013 2380 0 0.001929279999 2549 18 2844 858 0 0.002140050055 106 1716 590 2441 0 0.002977370052 1497 1861 2410 2477 0 0.004566189833 2182 1879 2672 2265 0 0.005298609845 1859 1353 2666 2253 0 0.006763460115 134 1516 885 2392 0 0.015571599826 1943 1915 2739 2356 0 0.034973099828 1759 1679 2694 2428 0 0.051208499819 161 1836 985 2513',
  '0 0.001050950028 1718 282 2147 801 0 0.001279830001 1875 441 2163 751 0 0.001294140005 734 974 1207 1320 0 0.001404760056 1054 528 1381 1151 0 0.001426700037 1739 1065 2566 1793 0 0.001428600051 1968 1074 2413 1429 0 0.001559259952 571 774 946 1424 0 0.001584050013 1718 924 2196 1525 0 0.002174380003 587 974 1332 1561 0 0.002939220052 853 792 1413 1438 0 0.004863739945 663 491 1060 1370 0 0.005016330164 1995 1010 2517 1647 0 0.005287169944 1767 555 2191 1402 0 0.005298609845 1837 360 2218 874 0 0.006031040102 968 355 1375 1329 0 0.008674619719 821 596 1315 1274 0 0.009109499864 2076 805 2457 1465 0 0.010925300419 1777 282 2245 1142 0 0.018096899614 625 901 1098 1461 0 0.041534401476 712 255 1326 1365 0 0.06671140343 1843 906 2435 1520 0 0.087585397065 1843 441 2408 1443 0 0.133056998253 674 751 1250 1402'],
 ['0 0.001069069956 2071 1432 2516 1903 0 0.001071930048 434 1432 1046 2205 0 0.001148220035 345 1981 946 2422 0 0.001200680039 278 1710 879 2477 0 0.001334190019 1637 1462 2321 2187 0 0.00133990997 2110 1909 2839 2537 0 0.001671790029 507 1631 1230 2441 0 0.002344130073 195 677 779 2410 0 0.002744670026 2199 1836 2744 2314 0 0.002750399988 345 574 1119 2404 0 0.002830510028 72 1571 662 2495 0 0.003356930101 1726 1263 2488 2006 0 0.003517149948 1776 834 1881 979 0 0.004467010032 779 598 1030 930 0 0.005176539999 1926 1969 2755 2368 0 0.00624846993 2087 1323 2611 2120 0 0.009170530364 2215 1981 2627 2241 0 0.010810899548 2204 918 2399 1166 0 0.017578100786 189 1335 1024 2465 0 0.0211334005 785 634 980 834 0 0.033538799733 1664 1800 2744 2573 0 0.035644501448 117 1855 1063 2555 0 0.065368697047 1765 1365 2672 2284',
  '0 0.00100230996 2136 1083 2413 1497 0 0.001049999963 777 505 1245 1206 0 0.001782419975 1837 582 2294 1265 0 0.002706530038 1729 892 2283 1506 0 0.002946849912 717 1033 1239 1393 0 0.005855559837 652 587 1076 1402 0 0.006282810122 625 942 1109 1443 0 0.006862639915 1957 1083 2446 1575 0 0.008918760344 908 710 1375 1406 0 0.011115999892 587 974 1348 1547 0 0.016037000343 1783 200 2359 1502 0 0.028732299805 1767 960 2511 1643 0 0.050811801106 756 173 1381 1433 0 0.13513199985 696 760 1272 1433 0 0.139403998852 1854 701 2419 1525'],
 ['0 0.001006130013 117 0 1247 278 0 0.00101757003 373 1661 918 2151 0 0.001114850049 612 1329 1136 2102 0 0.001125340001 623 1607 1163 2362 0 0.001224520034 1547 961 2416 2078 0 0.001236920012 746 580 996 864 0 0.001264570048 2071 1263 2755 2277 0 0.001321790041 0 0 1013 592 0 0.001451489981 273 2066 918 2513 0 0.00148392003 2176 1957 2722 2314 0 0.00182723999 779 344 1213 900 0 0.001972200116 1954 834 2444 1238 0 0.001985549927 301 930 1019 2120 0 0.002172470093 78 1861 674 2537 0 0.002866740106 1603 1414 2338 2187 0 0.003252029885 1982 1190 2549 1897 0 0.003602979938 6 2042 1024 2833 0 0.004020689987 1887 761 2561 1975 0 0.007167819887 1982 1782 2772 2441 0 0.008216859773 1742 1522 2711 2477 0 0.008255000226 401 1734 1102 2453 0 0.011337299831 484 1546 1024 2084 0 0.030761700124 273 1474 1035 2350 0 0.071838401258 1792 1244 2622 2114 0 0.09014890343 161 1903 1080 2586',
  '0 0.001395229949 1859 228 2207 664 0 0.001507760026 837 428 1305 1115 0 0.001551629975 1805 1028 2256 1406 0 0.00233649998 1881 1270 2533 1839 0 0.002340320032 1848 1051 2783 2321 0 0.002706530038 712 969 1158 1333 0 0.002719880082 870 109 1424 1160 0 0.002773280023 1859 655 2375 1356 0 0.002847670112 1951 1106 2403 1474 0 0.002912519965 1821 332 2272 1092 0 0.003772740019 598 856 962 1406 0 0.004974369891 1712 810 2267 1479 0 0.005367279984 799 646 1337 1356 0 0.005844119936 2022 805 2457 1479 0 0.00597763015 614 623 1087 1388 0 0.008445739746 576 1024 1141 1470 0 0.00851439964 2033 1060 2495 1620 0 0.008987429552 788 1028 1337 1474 0 0.01352689974 1674 1056 2462 1857 0 0.014770500362 1821 205 2180 482 0 0.037597700953 663 228 1348 1406 0 0.037811301649 1739 223 2381 1452 0 0.119079999626 647 828 1277 1429 0 0.153075993061 1815 983 2446 1565'],
 ['0 0.001167300041 0 2797 345 3087 0 0.001195910037 1765 973 2471 1873 0 0.001237869961 184 1009 1013 2223 0 0.001709940028 2260 1939 2733 2326 0 0.001749990042 1876 1957 2683 2314 0 0.001759530045 395 1963 991 2332 0 0.001902580028 1631 1269 2382 2048 0 0.002372740069 156 1740 696 2428 0 0.002603529952 2099 1051 2583 2000 0 0.003074649954 568 1607 1058 2181 0 0.004138949793 2021 1426 2483 1903 0 0.004631040152 2227 1963 2627 2169 0 0.00581741007 390 1414 963 2187 0 0.006683350075 1881 562 2538 2090 0 0.008979800157 2160 1414 2661 2151 0 0.009071350098 2093 1764 2672 2235 0 0.013450600207 1826 1625 2677 2308 0 0.013641400263 139 1963 1013 2489 0 0.035247799009 189 1595 985 2404 0 0.083984397352 1865 1269 2622 2102',
  '0 0.001200680039 669 1028 957 1402 0 0.001213070005 1805 282 2277 960 0 0.001418109983 1837 992 2316 1379 0 0.001681330032 745 801 1049 1283 0 0.001708979951 1848 592 2310 1256 0 0.002075199969 2087 805 2375 1265 0 0.002349850023 739 1001 1060 1356 0 0.002733229892 2076 1042 2392 1415 0 0.002761839889 924 1047 1272 1397 0 0.003011699999 1864 241 2158 555 0 0.003952030092 891 742 1359 1374 0 0.005191800185 788 928 1169 1311 0 0.005493159872 2022 592 2435 1374 0 0.007041930221 2055 906 2462 1497 0 0.008911130019 690 605 1114 1338 0 0.010368299671 783 1010 1326 1470 0 0.0128250001 641 956 1125 1429 0 0.014846799895 1826 1069 2446 1534 0 0.020080599934 1767 264 2392 1370 0 0.025146499276 745 282 1299 1352 0 0.143676996231 1881 815 2403 1443 0 0.159789994359 723 828 1250 1393']]

In [107]:
kfold_predictions[0]

['0 0.001079560025 1625 1891 2427 2416 0 0.001121519948 1937 1299 2394 1758 0 0.001185419969 384 501 1097 1655 0 0.001243590028 423 816 757 1214 0 0.001252169954 289 809 662 1365 0 0.001281740027 579 1909 1169 2416 0 0.001282689977 334 1317 774 1903 0 0.001399039989 2137 1389 2527 1843 0 0.001568789943 1915 1927 2705 2398 0 0.001731869997 2154 1305 2655 2090 0 0.001977920067 540 1631 1102 2344 0 0.002027509967 1648 1855 2739 2604 0 0.002063750057 106 1607 629 2453 0 0.002161029959 328 2380 1386 3093 0 0.002309799893 334 665 891 1516 0 0.002346039983 406 1504 640 1716 0 0.003021240002 1653 1250 2327 2030 0 0.003068919992 379 1389 946 2235 0 0.003293989925 262 761 718 1849 0 0.004394529853 2082 1172 2572 1879 0 0.004405979998 1870 731 2427 1432 0 0.004520419985 1648 683 2433 1957 0 0.004955289885 223 1861 1280 2984 0 0.008049010299 1753 1565 2627 2422 0 0.009658809751 217 1009 774 2332 0 0.010353099555 289 731 980 2205 0 0.01126859989 195 1909 1124 2580 0 0.017990099266 2132 870 2405 114

In [108]:
kfold_predictions[0][0]

'0 0.001079560025 1625 1891 2427 2416 0 0.001121519948 1937 1299 2394 1758 0 0.001185419969 384 501 1097 1655 0 0.001243590028 423 816 757 1214 0 0.001252169954 289 809 662 1365 0 0.001281740027 579 1909 1169 2416 0 0.001282689977 334 1317 774 1903 0 0.001399039989 2137 1389 2527 1843 0 0.001568789943 1915 1927 2705 2398 0 0.001731869997 2154 1305 2655 2090 0 0.001977920067 540 1631 1102 2344 0 0.002027509967 1648 1855 2739 2604 0 0.002063750057 106 1607 629 2453 0 0.002161029959 328 2380 1386 3093 0 0.002309799893 334 665 891 1516 0 0.002346039983 406 1504 640 1716 0 0.003021240002 1653 1250 2327 2030 0 0.003068919992 379 1389 946 2235 0 0.003293989925 262 761 718 1849 0 0.004394529853 2082 1172 2572 1879 0 0.004405979998 1870 731 2427 1432 0 0.004520419985 1648 683 2433 1957 0 0.004955289885 223 1861 1280 2984 0 0.008049010299 1753 1565 2627 2422 0 0.009658809751 217 1009 774 2332 0 0.010353099555 289 731 980 2205 0 0.01126859989 195 1909 1124 2580 0 0.017990099266 2132 870 2405 1148

In [129]:
# predictions = (model(fold), image, boxes)
# change = (image, model(fold), boxes)

kfold_id = kfold_ids[0]

boxes_list = []
scores_list = []
labels_list = []

# 각 모델이 image 당 예측한 것을 뽑아내기

# fold 순으로 하나씩 append

for i in range(len(kfold_id)):
    image_boxes = []
    image_scores = []
    image_labels = []
    
    for fold in range(5):
        # (image, boxes) <- each model
        temp_image = kfold_predictions[fold][i].split()
        boxes = []
        scores = []
        labels = []
        for j in range(len(temp_image)//6):
            temp = temp_image[j*6:j*6+6]
            labels.append(int(temp[0]))
            scores.append(float(temp[1]))
            boxes.append(list(map(int,temp[2:])))

        image_boxes.append(boxes)
        image_scores.append(scores)
        image_labels.append(labels)
    
    boxes_list.append(image_boxes)
    scores_list.append(image_scores)
    labels_list.append(image_labels)

In [150]:
print('image :',len(boxes_list))
print('fold  :',len(boxes_list[0]))
print('num boxes')
for i in boxes_list[0]:
    print(len(i))

image : 2
fold  : 5
num boxes
31
10
23
25
20


In [159]:
wbf_boxes_list = []
wbf_scores_list = []
wbf_labels_list = []

for boxes, scores, labels in zip(boxes_list, scores_list, labels_list): # by image

    
    boxes, scores, labels = weighted_boxes_fusion(boxes, 
                                                  scores, 
                                                  labels, 
                                                  weights=None, 
                                                  iou_thr=0.9, 
                                                  skip_box_thr=0.0, 
                                                  conf_type='avg', 
                                                  allows_overflow=False)
    
    wbf_boxes_list.append(boxes)
    wbf_scores_list.append(scores)
    wbf_labels_list.append(labels)

In [210]:
# Ah. weighted_boxes_fusion 은 [0,1] 사이의 비율로 float 받는데, 나는 결론이 난 결과물을 들이밀고 있었다.
# 그러니 결과가 안나오지... 잘 읽어보자...
wbf_boxes_list

[array([[0., 0., 1., 1.]]), array([], shape=(0, 4), dtype=float64)]

In [167]:
kfold_df.iloc[0:2,:]

,id,PredictionString
0,51759b5579bc_image,0 0.001079560025 1625 1891 2427 2416 0 0.00112...
1,65761e66de9f_image,0 0.001008030027 1854 218 2180 496 0 0.0010547...


In [180]:
kfold_df = pd.read_csv("./kfold_df.csv")
folds = 5
subs = [kfold_df.iloc[i*2:i*2+2].reset_index(drop=True) for i in range(folds)]

### Reference code

Notebook link : https://www.kaggle.com/morizin/wbf-ensemble

In [333]:
from glob import glob

data = glob('./dummy/kfold_result_dir/*')
data

['./dummy/kfold_result_dir/kfold_0_yolo.csv',
 './dummy/kfold_result_dir/kfold_1_yolo.csv',
 './dummy/kfold_result_dir/kfold_df_yolo.csv',
 './dummy/kfold_result_dir/meta.csv',
 './dummy/kfold_result_dir/kfold_3_yolo.csv',
 './dummy/kfold_result_dir/kfold_2_yolo.csv',
 './dummy/kfold_result_dir/kfold_4_yolo.csv']

In [334]:
height_dict = pd.read_csv('./dummy/kfold_result_dir/meta.csv')
height_dict.columns = ['image_id','width','height','split']
height_dict = height_dict[['image_id','width','height']]
height_dict = height_dict.to_dict('records')
# to_dict('records') : 기록처럼 list 화

In [335]:
height_dict

[{'image_id': '65761e66de9f_image', 'width': 2330, 'height': 2783},
 {'image_id': '51759b5579bc_image', 'width': 3093, 'height': 2850}]

In [336]:
fnl_dict = {}
for ix, i in enumerate(height_dict):
    fnl_dict[i['image_id']] = [i['width'],i['height'],i['width'],i['height']]
fnl_dict

{'65761e66de9f_image': [2330, 2783, 2330, 2783],
 '51759b5579bc_image': [3093, 2850, 3093, 2850]}

In [337]:
subs = [
pd.read_csv('./dummy/kfold_result_dir/kfold_0_yolo.csv'),
pd.read_csv('./dummy/kfold_result_dir/kfold_1_yolo.csv'),
pd.read_csv('./dummy/kfold_result_dir/kfold_3_yolo.csv'),
pd.read_csv('./dummy/kfold_result_dir/kfold_2_yolo.csv'),
pd.read_csv('./dummy/kfold_result_dir/kfold_4_yolo.csv')]

In [338]:
# sample
pd.read_csv('./dummy/kfold_result_dir/kfold_0_yolo.csv')

,id,PredictionString
0,51759b5579bc_image,0 0.001079560025 1625 1891 2427 2416 0 0.00112...
1,65761e66de9f_image,0 0.001008030027 1854 218 2180 496 0 0.0010547...


In [339]:
def submission_decoder(df:pd.DataFrame) -> np.ndarray:
    info = df.values
    df_lst = []
    for i in info:
        pre_lst = i[1].split(' ')
        for j in range(0, len(pre_lst), 6):
            df_lst.append([i[0], int(pre_lst[j]), float(pre_lst[j+1]),
                           int(pre_lst[j+2]), int(pre_lst[j+3]),
                           int(pre_lst[j+4]), int(pre_lst[j+5]),
                           fnl_dict.get(i[0])[0], fnl_dict.get(i[0])[1]])
    return pd.DataFrame(df_lst, columns = ['image_id', 'class_id', 'score',
                                           'x_min','y_min','x_max','y_max',
                                           'width','height'])

In [340]:
subs = [submission_decoder(subs[i]) for i in range(len(subs))]

In [341]:
boxes_dict = {}
scores_dict = {}
labels_dict = {}
whwh_dict = {}

for i in tqdm(subs[0].image_id.unique()):
    if not i in boxes_dict.keys():
        boxes_dict[i] = []
        scores_dict[i] = []
        labels_dict[i] = []
        whwh_dict[i] = []
        
    size_ratio = fnl_dict.get(i)
    whwh_dict[i].append(size_ratio)
    tmp_df = [subs[x][subs[x]['image_id']==i] for x in range(len(subs))]
    
    for x in range(len(tmp_df)):
        boxes_dict[i].append(((tmp_df[x][['x_min','y_min','x_max','y_max']].values)/size_ratio).tolist())
        scores_dict[i].append(tmp_df[x]['score'].values.tolist())
        labels_dict[i].append(tmp_df[x]['class_id'].values.tolist())

100%|██████████| 2/2 [00:00<00:00, 151.95it/s]


In [342]:
weights = [1] * 5
iou_thr = 0.1
skip_box_thr = 0.0
sigma = 0.1

fnl = {}

for i in tqdm(boxes_dict.keys()):
    boxes, scores, labels = weighted_boxes_fusion(boxes_dict[i], 
                                                  scores_dict[i], 
                                                  labels_dict[i],
                                                  weights=weights,
                                                  iou_thr=iou_thr,
                                                  skip_box_thr=skip_box_thr)
    if not i in fnl.keys():
        fnl[i] = {'boxes':[],
                  'scores':[],
                  'labels':[]}
    fnl[i]['boxes'] = boxes * whwh_dict[i]
    fnl[i]['scores'] = scores
    fnl[i]['labels'] = labels

  0%|          | 0/2 [00:00<?, ?it/s]/Users/madquer/opt/anaconda3/envs/data/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/Users/madquer/opt/anaconda3/envs/data/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
100%|██████████| 2/2 [00:00<00:00, 40.97it/s]


In [343]:
fnl

{'51759b5579bc_image': {'boxes': array([[ 210.33032566, 1679.76399064, 1016.00985736, 2466.04010761],
         [1855.6100015 , 1400.71262866, 2625.15239453, 2187.25230396],
         [2111.33348608,  865.32374024, 2408.09800994, 1194.14634705],
         [ 781.99959111,  607.57350549, 1003.33502564,  854.46213037],
         [ 290.57929485,  807.93886185,  842.25571695, 1848.41159284],
         [1776.        ,  834.        , 1881.        ,  979.        ],
         [ 406.        , 1504.        ,  640.        , 1716.        ],
         [  50.56755183,    0.        , 1114.13515744,  456.28881678],
         [ 328.        , 2380.        , 1386.        , 2850.        ],
         [2549.        ,   18.        , 2844.        ,  858.        ],
         [   0.        , 2797.        ,  345.        , 2850.        ]]),
  'scores': array([0.01185822, 0.01163729, 0.00703584, 0.00573291, 0.0017928 ,
         0.00070343, 0.00046921, 0.00046558, 0.00043221, 0.00038586,
         0.00023346]),
  'labels': arr

In [344]:
pd_form = []
for i in fnl.keys():
    b = fnl[i] # image_id
    for j in range(len(b['boxes'])):
        pd_form.append([i, 
                        int(b['labels'][j]), float(b['scores'][j]),
                        int(b['boxes'][j][0]), int(b['boxes'][j][1]),
                        int(b['boxes'][j][2]), int(b['boxes'][j][3])])
final_df = pd.DataFrame(pd_form, columns = ['image_id', 'class_id', 'score',
                                            'x_min', 'y_min' ,'x_max', 'y_max'])
final_df = final_df.drop_duplicates(keep = 'first')
        
# 왜 confidence score 를 round 처리 하지? 하고 안하고 점수 차이를 비교해봐야겠다.

In [345]:
def submission_encoder(df:pd.DataFrame) -> np.ndarray:
    dct = {}
    for i in tqdm(df['image_id'].unique()):
        if not i in dct.keys():
            dct[i] = []
        tmp = df[df['image_id']==i].values
        for j in tmp:
            dct[i].append(int(j[1]))
            dct[i].append(float(j[2]))
            dct[i].append(int(j[3]))
            dct[i].append(int(j[4]))
            dct[i].append(int(j[5]))
            dct[i].append(int(j[6]))
        dct[i] = map(str, dct[i])
        dct[i] = ' '.join(dct[i])
    dct = [[k,v] for k, v in dct.items()]
    return pd.DataFrame(dct, columns = ['image_id', 'PredictionString']).reset_index(drop=True)

df = submission_encoder(final_df)
df

100%|██████████| 2/2 [00:00<00:00, 1370.02it/s]


,image_id,PredictionString
0,51759b5579bc_image,0 0.011858216486871243 210 1679 1016 2466 0 0....
1,65761e66de9f_image,0 0.023412512615323067 1856 749 2327 1492 0 0....


In [346]:
df.loc[0, 'PredictionString']

'0 0.011858216486871243 210 1679 1016 2466 0 0.011637290939688683 1855 1400 2625 2187 0 0.007035835646092892 2111 865 2408 1194 0 0.005732913967221975 781 607 1003 854 0 0.0017928010784089565 290 807 842 1848 0 0.0007034299896 1775 834 1881 978 0 0.0004692079966 406 1504 640 1716 0 0.0004655839875340462 50 0 1114 456 0 0.00043220599180000005 328 2380 1386 2850 0 0.0003858559998 2549 18 2844 858 0 0.0002334600082 0 2797 345 2850'

In [347]:
df.loc[0, 'PredictionString']

'0 0.011858216486871243 210 1679 1016 2466 0 0.011637290939688683 1855 1400 2625 2187 0 0.007035835646092892 2111 865 2408 1194 0 0.005732913967221975 781 607 1003 854 0 0.0017928010784089565 290 807 842 1848 0 0.0007034299896 1775 834 1881 978 0 0.0004692079966 406 1504 640 1716 0 0.0004655839875340462 50 0 1114 456 0 0.00043220599180000005 328 2380 1386 2850 0 0.0003858559998 2549 18 2844 858 0 0.0002334600082 0 2797 345 2850'

In [348]:
more = False
if more:
    NORMAL = "14 1 0 0 1 1"
    low_threshold = 0.00
    high_threshold = 0.99
    pred_det_df = df  # You can load from another submission.csv here too.
    n_normal_before = len(pred_det_df.query("PredictionString == @NORMAL"))
    merged_df = pd.merge(pred_det_df, pred_2cls, on="image_id", how="left")

    if "target" in merged_df.columns:
        merged_df["class0"] = 1 - merged_df["target"]

    c0, c1, c2 = 0, 0, 0
    for i in range(len(merged_df)):
        p0 = merged_df.loc[i, "class0"]
        if p0 < low_threshold:
            # Keep, do nothing.
            c0 += 1
        elif low_threshold <= p0 and p0 < high_threshold:
            # Add, keep "det" preds and add normal pred.
            if ' 14 ' not in merged_df.loc[i, "PredictionString"]:
                merged_df.loc[i, "PredictionString"] += f" 14 {p0} 0 0 1 1"

            c1 += 1
        else:
            # Replace, remove all "det" preds.
            merged_df.loc[i, "PredictionString"] = NORMAL
            c2 += 1

    n_normal_after = len(merged_df.query("PredictionString == @NORMAL"))
    print(
        f"n_normal: {n_normal_before} -> {n_normal_after} with threshold {low_threshold} & {high_threshold}"
    )
    print(f"Keep {c0} Add {c1} Replace {c2}")
    submission_filepath = str("submission.csv")
    submission_df = merged_df[["image_id", "PredictionString"]]
    submission_df.to_csv(submission_filepath, index=False)
    print(f"Saved to {submission_filepath}")